In [ ]:
from src.data import Data
import matplotlib.pyplot as plt
%matplotlib inline
data = Data()

In [ ]:
def get_sample(data, defect_id):
    for i_n in data.image_names:
        i_defects = data.get_image_annotations(i_n)
        if i_defects[defect_id]:
            image = data.load_image(i_n)
            mask = data.get_defects_masks(i_n)[defect_id]
            return image, mask
        
img, bmp = get_sample(data, 4)
print(img.shape)
plt.figure(figsize=(20,20))
plt.imshow(bmp)
plt.figure(figsize=(20,20))
plt.imshow(img)